# Kaggle API Setup for TensorFlow and Keras NLP Project
Gemma is a family of lightweight, state-of-the-art open models built from the same research and technology used to create the Gemini models.

## Introduction
In this notebook, we will prepare the environment to download and utilize the Gemma model in a TensorFlow and Keras NLP project. We will begin by importing the necessary libraries, followed by configuring the Kaggle API credentials to enable seamless access to the required datasets.

## 1. Importing Required Libraries
To start, we will import the essential libraries for this project, including TensorFlow, Keras, and Keras NLP, which are crucial for building and deploying NLP models.


In [ ]:
# Import required libraries

import os
import tensorflow as tf
import keras_nlp
import keras
import json

# Ignore Warnings
from silence_tensorflow import silence_tensorflow
silence_tensorflow()
import warnings
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [2]:
# Set Kaggle API credentials

os.environ["KAGGLE_USERNAME"] = "rogerkorantenng"
os.environ["KAGGLE_KEY"] = "9a33b6e88bcb6058b1281d777fa6808d"

## 2. Building and Compiling the Gemma Model

In this section, we will build and compile the Gemma model, a language model designed for natural language processing (NLP) tasks. The process involves several key steps: loading the pre-trained model, enabling fine-tuning with LoRA (Low-Rank Adaptation), configuring the input sequence length, setting up the optimizer, and compiling the model for training.


In [3]:
def get_compiled_model():
    gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
    gemma_lm.summary()

    gemma_lm.backbone.enable_lora(rank=4)
    gemma_lm.summary()
    
    # Set the sequence length to 128 before using the model.
    gemma_lm.preprocessor.sequence_length = 256
    
    # Use AdamW (a common optimizer for transformer models).
    optimizer = keras.optimizers.AdamW(
        learning_rate=5e-5,
        weight_decay=0.01,
    )
    
    # Exclude layernorm and bias terms from decay.
    optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])
    
    gemma_lm.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=optimizer,
        weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )

    
    return gemma_lm

## 3. Loading and Processing the Dataset

In this section, we will define a function to load and process a JSON dataset. The dataset is read line by line, and each line is parsed and formatted according to the required structure. The function returns a list of formatted examples that can be used for training or analysis.


In [4]:
def get_dataset():
    # Initialize an empty list to hold the processed data.
    data = []
    
    # Open and read the JSON file line by line.
    with open('/project/data/combined_dataset.json') as file:
        for line in file:
            features = json.loads(line)
            
            # Filter out examples without "Context".
            if not features.get("Context"):
                continue
            
            # Format the example as a string.
            template = "Instruction:\n{Context}\n\nResponse:\n{Response}"
            formatted_example = template.format(**features)
            
            # Append the formatted example to the data list.
            data.append(formatted_example)
    
    return data 

In [5]:
# Get Model and Compile
model  = get_compiled_model()

# Get the dataset outside the strategy scope.
data = get_dataset()

2024-08-30 16:13:39.953235: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13775 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:18:00.0, compute capability: 7.5
2024-08-30 16:13:39.954747: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13775 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:19:00.0, compute capability: 7.5
2024-08-30 16:13:39.956103: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 13775 MB memory:  -> device: 2, name: Tesla T4, pci bus id: 0000:35:00.0, compute capability: 7.5
2024-08-30 16:13:39.957459: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 13775 MB memory:  -> device: 3, name: Tesla T4, pci bus id: 0000:36:00.0, compute capability: 7.5
2024-08-

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

...

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

## 4. Defining the Prompt Template and Generating Responses

In this section, we define a template for creating prompts that the language model will use to generate responses. The template includes placeholders for an 'instruction' and a 'response'. We then format this template with actual data to create a complete prompt. Finally, the prompt is passed to the language model to generate a response.


In [6]:
# Define the template with placeholders for 'instruction' and 'response'
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"

# Create the prompt by formatting the template with actual data
prompt = template.format(
    instruction="I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
    response="",
)

# Assuming gemma_lm is a language model that you're using to generate text
print(model.generate(prompt, max_length=256))


2024-08-30 16:13:57.828298: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT64 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.
I0000 00:00:1725034446.797146   18073 service.cc:146] XLA service 0x5a7be0cb31d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1725034446.797176   18073 service.cc:154]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1725034446.797180   18073 service.cc:154]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1725034446.797182   18073 service.cc:154]   StreamExecutor device (2): Tesla T4, Compute Capability 7.5
I0000 00:00:1725034446.797185   18073 service.cc:154]   StreamExecutor device (3): Tesla T4, Compute Capability 7.5
I0000 00:00:1725034446.797188   18073 service.cc:154]   StreamExecutor device (4): Tesla T4, Compute Capability 7.5
I0000 00:00:1725034446.797190   18073 service.cc:154]   StreamE

Instruction:
I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?

Response:
I'm sorry to hear that you're going through some things. I'm not sure what you mean by "I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here." I'm not sure what you mean by "I've never tried or contemplated suicide." I'm not sure what you mean by "I've always wanted to fix my issues, but I never get around to it." I'm not sure what you mean by "How can I change my feeling of being worthless to everyone?"

I'm not sure what you mean by "I'm sorry to hear that you're going through some things."

I'm not sure what you mean by "I'm not sure what you mean by 'I barely sleep


## 5. Model Fine Tunning

In this section, we compile the model, prepare the dataset, and then train the model using the data. We will walk through the steps of obtaining the compiled model, loading the dataset, and fitting the model to the data.


In [7]:
# Fit the model using the data.
model.fit(data, epochs=40, batch_size=1, verbose=1)

Epoch 1/40


W0000 00:00:1725034506.217331   18546 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
2024-08-30 16:15:23.759806: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_2', 220 bytes spill stores, 220 bytes spill loads



3512/3512 ━━━━━━━━━━━━━━━━━━━━ 2455s 682ms/step - loss: 2.0513 - sparse_categorical_accuracy: 0.4572
Epoch 2/40
3512/3512 ━━━━━━━━━━━━━━━━━━━━ 2398s 683ms/step - loss: 1.9421 - sparse_categorical_accuracy: 0.4760
Epoch 3/40
3512/3512 ━━━━━━━━━━━━━━━━━━━━ 2398s 683ms/step - loss: 1.8639 - sparse_categorical_accuracy: 0.4929
Epoch 4/40
3512/3512 ━━━━━━━━━━━━━━━━━━━━ 2396s 682ms/step - loss: 1.7974 - sparse_categorical_accuracy: 0.5083
Epoch 5/40
3512/3512 ━━━━━━━━━━━━━━━━━━━━ 2397s 683ms/step - loss: 1.7301 - sparse_categorical_accuracy: 0.5239
Epoch 6/40
3512/3512 ━━━━━━━━━━━━━━━━━━━━ 2396s 682ms/step - loss: 1.6699 - sparse_categorical_accuracy: 0.5380
Epoch 7/40
3512/3512 ━━━━━━━━━━━━━━━━━━━━ 2397s 682ms/step - loss: 1.6151 - sparse_categorical_accuracy: 0.5512
Epoch 8/40
3512/3512 ━━━━━━━━━━━━━━━━━━━━ 2399s 683ms/step - loss: 1.5641 - sparse_categorical_accuracy: 0.5633
Epoch 9/40
3512/3512 ━━━━━━━━━━━━━━━━━━━━ 2393s 681ms/step - loss: 1.5194 - sparse_categorical_accuracy: 0.5776
Epo

## 6. Generating a Response from the Fine-Tuned Language Model

In this section, we will define a template for generating prompts and use the language model `gemma_lm` to generate a response based on the provided instruction. This process involves creating a formatted prompt and then using the model to produce a response.


In [9]:
# Define the template with placeholders for 'instruction' and 'response'
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"

# Create the prompt by formatting the template with actual data
prompt = template.format(
    instruction="I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
    response="",
)

# Assuming gemma_lm is a language model that you're using to generate text
print(model.generate(prompt, max_length=256))


Instruction:
I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?

Response:
It sounds like you are having a really tough time with feeling this way.  Feeling this way is not normal and it is important to talk about these feelings with someone.  You are not worthless, you are a wonderful person who is going through some tough times.  Therapy can help you to work through these issues and come to a place of self-love and acceptance.  You can do this!  


In [ ]:
# Fit the model using the data.
model.fit(data, epochs=90, batch_size=1, verbose=1)

Epoch 1/90
3512/3512 ━━━━━━━━━━━━━━━━━━━━ 2388s 680ms/step - loss: 0.8785 - sparse_categorical_accuracy: 0.7438
Epoch 2/90
3512/3512 ━━━━━━━━━━━━━━━━━━━━ 2388s 680ms/step - loss: 0.8661 - sparse_categorical_accuracy: 0.7473
Epoch 3/90
3512/3512 ━━━━━━━━━━━━━━━━━━━━ 2384s 679ms/step - loss: 0.8522 - sparse_categorical_accuracy: 0.7514
Epoch 4/90
3512/3512 ━━━━━━━━━━━━━━━━━━━━ 2380s 678ms/step - loss: 0.8408 - sparse_categorical_accuracy: 0.7547
Epoch 5/90
3512/3512 ━━━━━━━━━━━━━━━━━━━━ 2379s 677ms/step - loss: 0.8266 - sparse_categorical_accuracy: 0.7585
Epoch 6/90
3512/3512 ━━━━━━━━━━━━━━━━━━━━ 2380s 678ms/step - loss: 0.8117 - sparse_categorical_accuracy: 0.7635
Epoch 7/90
3512/3512 ━━━━━━━━━━━━━━━━━━━━ 2379s 677ms/step - loss: 0.8048 - sparse_categorical_accuracy: 0.7653
Epoch 8/90
3512/3512 ━━━━━━━━━━━━━━━━━━━━ 2382s 678ms/step - loss: 0.7892 - sparse_categorical_accuracy: 0.7698
Epoch 9/90
3512/3512 ━━━━━━━━━━━━━━━━━━━━ 2387s 680ms/step - loss: 0.7790 - sparse_categorical_accuracy:

In [20]:
model.backbone.save_lora_weights("model2.lora.h5")

In [23]:
# Define the template with placeholders for 'instruction' and 'response'
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"

# Create the prompt by formatting the template with actual data
prompt = template.format(
    instruction="I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
    response="",
)

# Assuming gemma_lm is a language model that you're using to generate text
print(model.generate(prompt, max_length=256))


Instruction:
I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.
   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.
   How can I change my feeling of being worthless to everyone?

Response:
I'm glad you're willing to slow down, but it's a complicated feeling that can be challenging to identify and work through. I've heard it described as feeling like a heavy cloud or rain cloud, and it's good to know where you are when you want to know what to do about it.It is possible to know more about this feeling and why it's arising from time to time like this. It helps to know that what you're experiencing is likely to do with your sense of who you are and your sense of how important you are in people's lives. It's also helpful to know that this feeling is something that's happening deep within you, not something outside of you.It can 

In [ ]:
model.fit(data, epochs=1, batch_size=1, verbose=1)

3136/3512 ━━━━━━━━━━━━━━━━━━━━ 4:11 668ms/step - loss: 0.3683 - sparse_categorical_accuracy: 0.8940